In [2]:
%pip install pyriksdagen
from lxml import etree
import progressbar
from pyparlaclarin.read import paragraph_iterator, speeches_with_name
from pyriksdagen.utils import protocol_iterators, download_corpus
import pyriksdagen
# We need a parser for reading in XML data
parser = etree.XMLParser(remove_blank_text=True)
import pandas as pd
from sklearn.model_selection import train_test_split

Note: you may need to restart the kernel to use updated packages.


In [2]:
print(pyriksdagen.__spec__)
download_corpus(partitions=["politicians", "records"])

ModuleSpec(name='pyriksdagen', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7473401ad5d0>, origin='/home/laurinemeier/anaconda3/lib/python3.11/site-packages/pyriksdagen/__init__.py', submodule_search_locations=['/home/laurinemeier/anaconda3/lib/python3.11/site-packages/pyriksdagen'])


politicians.zip: 100%|██████████| 2.20M/2.20M [00:00<00:00, 28.9MiB/s]

11:11:34 [WARNING] (pyriksdagen): data already exists at the path 'data'. It will be overwritten once the download is finished.



records.zip: 100%|██████████| 1.60G/1.60G [00:20<00:00, 84.8MiB/s]


In [3]:
protocols = list(protocol_iterators(corpus_root="data/", start=1867, end=202122))

In [4]:
def oppna_data_to_dict(input_dict):
    """
    Load protocols with the new XML / HTML structure (from 2013 onwards)
    and convert it to a python dict with contents.
    """
    data = {}
    data["paragraphs"] = []

    # Metadata
    session = input_dict["dokumentstatus"]["dokument"]["rm"]
    session = session.replace("/", "")
    pid = input_dict["dokumentstatus"]["dokument"]["nummer"]
    date = input_dict["dokumentstatus"]["dokument"]["datum"]
    html = input_dict["dokumentstatus"]["dokument"]["html"]
    html_tree = clean_html(html)
    year = int(date.split("-")[0])
    protocol_id = f"prot-{session}--{pid}"

    data["protocol_id"] = protocol_id
    data["date"] = date.split(" ")[0]
    data["session"] = session

    # New HTML structure with div[@class='Section1']
    section1 = html_tree.xpath(".//div[@class='Section1']")
    for elements in section1:
        for elem in elements:
            if elem.tag in ["p", "h1", "h2"]:
                elemtext = "".join(elem.itertext())
                linebreak = elemtext.strip() == "" and "\n" in elemtext
                if linebreak:
                    pass
                else:
                    paragraph = elemtext.strip()
                    paragraph = paragraph.replace("\n", " ")
                    paragraph = re.sub("\\s+", " ", paragraph)
                    data["paragraphs"].append(paragraph)

    if len(data["paragraphs"]) == 0:
        tree = html_tree

        # Old data structure 1990-2003
        pres = tree.findall(".//pre")
        if len(pres) > 0:
            for pre in pres:
                if pre.text is not None:
                    tblocks = re.sub("([a-zß-ÿ,])- ?\n ?([a-zß-ÿ])", "\\1\\2", pre.text)
                    tblocks = re.sub("([a-zß-ÿ,]) ?\n ?([a-zß-ÿ])", "\\1 \\2", tblocks)
                    for paragraph in tblocks.split("\n"):
                        paragraph = paragraph.replace("\n", " ")
                        paragraph = paragraph.replace("\n", " ")
                        data["paragraphs"].append(paragraph)

        # Standard HTML structure, roughly 2003-2013
        elif len(tree.xpath("//div[@class='indrag']")) > 0:
            tree = tree.xpath("//body")[0]
            for elem in tree:
                elemtext = "".join(elem.itertext())
                linebreak = elemtext.strip() == "" and "\n" in elemtext
                if elem.tag == "br" or linebreak:
                    pass
                else:
                    paragraph = elemtext.strip()
                    paragraph = paragraph.replace("\n", " ")
                    paragraph = re.sub("\\s+", " ", paragraph)
                    data["paragraphs"].append(paragraph)
    return data


In [4]:
protocol_in_question = protocols[12]
root = etree.parse(protocol_in_question, parser).getroot()

In [13]:

for elem in list(paragraph_iterator(root, output="lxml"))[:7]:
  print(" ".join(elem.itertext()))



          RIKSDAGENS Ar PROTOKOLL
        

          1955 ANDRA KAMMAREN Nr 13
        

          13—15 april
        

          Debatter m. m.
        

          Onsdagen den 13 april Sid.
        

          Familjerådgivning «... 5 Interpellation av herr Ericsson : i
          Näs ang. de minskade perioderna
        


In [3]:
data=[]

for i in range(len(protocols)):
  protocol_in_question = protocols[i]
  root = etree.parse(protocol_in_question, parser).getroot()
  element_str=""
  for elem in list(paragraph_iterator(root, output="lxml")):
    element_str += " ".join(elem.itertext()).replace("\n","")
    
  data.append({"protocole": i,"texte": "".join(element_str.split())})

df=pd.DataFrame(data)
print(df)


       protocole                                              texte
0              0  Sedan,ikraftafRiketsRegeringsform,lagtimaRiksd...
1              1  24Den19Januari.Lördagenden19Januari,KL!/,11f.m...
2              2  Den21Januari.25mandeaftidenfördessaval,erhålli...
3              3  Den22Januari.41Tisdagenden22Januari.Kl.10£m.g1...
4              4  Den23Januari.-55Onsdagenden23Januari.Kl.10f.m....
...          ...                                                ...
17637      17637  §1JusteringavprotokollProtokolletförden7juniju...
17638      17638  §1JusteringavprotokollProtokolletförden8juniju...
17639      17639  §1JusteringavprotokollProtokollenförden9,10,13...
17640      17640  §1AnmälanomåtertagandeavplatsiriksdagenTalmann...
17641      17641  §1AnmälanomsubsidiaritetsprövningarTalmannenan...

[17642 rows x 2 columns]


In [4]:
df.to_pickle("swerick_data_long.pkl")

In [21]:
from datasets import load_dataset
swerick_dataset = load_dataset("pandas",data_files="swerick_data.pkl")

Generating train split: 0 examples [00:00, ? examples/s]

In [22]:
print(swerick_dataset)

DatasetDict({
    train: Dataset({
        features: ['protocole', 'texte'],
        num_rows: 130
    })
})


In [5]:
df_train,df_test = train_test_split(df,test_size=0.2,random_state=42)
df_train.to_pickle("swerick_data_long_train.pkl")
df_test.to_pickle("swerick_data_long_test.pkl")

In [13]:
#datasest
data_files = {"train": "swerick_data_train.pkl", "test": "swerick_data_test.pkl"}
swerick_dataset = load_dataset("pandas",data_files=data_files)
print(swerick_dataset)


DatasetDict({
    train: Dataset({
        features: ['protocole', 'texte', '__index_level_0__'],
        num_rows: 104
    })
    test: Dataset({
        features: ['protocole', 'texte', '__index_level_0__'],
        num_rows: 26
    })
})


In [5]:
protocol_in_question

'data/1867/prot-1867--ak--0202.xml'

In [22]:
import pandas as pd
from bs4 import BeautifulSoup



notes=[]
intros=[]

for i in range(len(protocols)):
    protocol_in_question = protocols[i]
    with open(protocol_in_question,"r") as f:
        xml_content=f.read()
    soup=BeautifulSoup(xml_content,"xml")
    note_elements=soup.find_all("note")

    for note in note_elements:
        note_text=note.text.strip()
        notes.append(note_text)

        next_element=note.get("type")
        if next_element == "speaker" :
            intros.append(True)
        else :
            intros.append(False)
 
df=pd.DataFrame({"Note" :notes, "Intro":intros})

print(df)

                                                      Note  Intro
0        Sedan, i kraft af Rikets Regeringsform, lagtim...  False
1           Fredagen den 18 Januari 1867, kl. '/, 11 f. m.  False
2        Riksdagens Andra Kammare uti den för dess samm...  False
3                                             Mine Herrar!  False
4        Enligt 33 8 i Riksdags-ordningen skall, innan ...  False
...                                                    ...    ...
7027171  § 7 Bordläggning och beslut om förlängd motion...  False
7027172          § 8 Anmälan om frågor för skriftliga svar  False
7027173           § 9 Anmälan om skriftliga svar på frågor  False
7027174  § 10 Anmälan om uteblivna svar på skriftliga f...  False
7027175                 § 11 Kammaren åtskildes kl. 11.40.  False

[7027176 rows x 2 columns]


In [23]:
df_train_intro,df_test_intro = train_test_split(df,test_size=0.2,random_state=42)
df_train_intro.to_pickle("swerick_data_intro_train.pkl")
df_test_intro.to_pickle("swerick_data_intro_test.pkl")